<a href="https://colab.research.google.com/github/tpet93/CUDA-linkage-disequilibrium/blob/master/CUDA-linkage-disequilibrium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Made to replicate the results of
allel.rogers_huff_r() in scikit-allel.


Modifications include returning r^2 and the option to return a running mean.
This second option allows for arbtrarilly large dataset at the cost of being able to calculate std-dev and similar parameters on the output.

Input is
Diploid genotypes at biallelic variants, coded as the number of alternate alleles per call (i.e., 0 = hom ref, 1 = het, 2 = hom alt).



In [0]:
# Run this to find out how much GPU Memory you have been allocated, this lets us use as much as possible.
!nvidia-smi

In [0]:
import torch
from tqdm.notebook import tqdm
# from tqdm import tqdm

from numba import jit
import time



In [0]:
# returns each r for a given set of pairs
def r_torch(x,y,cudatype):    

    #incase single pair is passed
    if len(x.shape)== 1:
        x = x.unsqueeze(dim = 0)
        y = y.unsqueeze(dim = 0)
    
    # we want to consider negative values as missing.
    # generate a mask of all non missing pairs
    mask = (x>=0) & (y>=0)


    #if we remove missing values from our tensor with the mask them we loose our array dimensions.

    #we can expoit the fact that the absolute values of our numbers do not affect our correlation coeficient implementation.
    #therefore: we can shift our inputs by + 1, this way missing pairs can be set to zero and not affect our sums.

    #shift all by one to make room for missing avlues to be zero
    x= x+1 
    y= y+1
    
    #set both elements to zero if one or both are missing.
    x[~mask]= 0 
    y[~mask]= 0
    
    #x - mean of x (ignoring missing values)
    vx = x.T-(torch.sum(x,dim =1).type(cudatype)/torch.sum(mask,dim =1))
    vy = y.T-(torch.sum(y,dim =1).type(cudatype)/torch.sum(mask,dim =1))

    #transpose
    vx = vx.T
    vy = vy.T

    #keep missing values at zero
    vx[~mask] = 0
    vy[~mask] = 0

    #calculate covariance
    cov = torch.sum(torch.mul(vx,vy),dim = 1)


    svx2 = torch.sqrt(torch.sum(vx ** 2,dim =1))
    svy2 = torch.sqrt(torch.sum(vy ** 2,dim =1))
    
    #Correlation Coefficient
    r = torch.div(cov,torch.mul(svx2,svy2))
  
    #Return r
    return(r)





In [0]:
'''
The below functions replicates torch.combinations()
however it is written in a way that combinations can be retrieved form any theoretical position and length without need to calcuate the entire tensor as it may be billions long.
combinations = makecombs(n)
is equivalent to
combinations = torch.combinations(torch.arange(n))
'''

#block number is the first member
@jit(nopython=True)
def getblocknumbyidx(n,idx):
    n2 = n - .5
    blocknum = ((n2) - np.sqrt((n2**2)-2*idx))
    return blocknum

@jit(nopython=True) 
def blocksize(n,blockn):
    return n-1-blockn

#makecombs(500,1000,2000)
#returns torch.combinations(torch.arange(500))[1000,2000]

@jit(nopython=True) 
def makecombs(n,starti = 0,endi = 0):
    size = n*(n-1)//2# size of all pairs
    if endi == 0:
        endi = size
    endi = min(endi,size)
    #get request output size and init tensor of zeros
    out = np.zeros((endi-starti,2))
    # each requested output pair calcuate foirst and second members
    for i in range(starti,endi):
        b = getblocknumbyidx(n,i)#block number is the first member
        bn = b//1 #reduce to floor integer

        res = b % 1 # get leftover residual after removing integer value
        #this residual represents the proportion of the second value along its range.
        #i,e 0.99 would indicate the second member is close to its maximum value for this block
        bindex = (res*blocksize(n,bn)+bn+1.5)//1
        #the math is a bit weird here but this effectivly converts the residual into the second member.
        out[i-starti] = bn,bindex

    return out

In [0]:
'''
batch_r2 splits the input data into batches to fit on a given amount of GPU RAM
fp16 is a bit faster but results differ slightly from fp32
ret_mean tracks a running mean of each batch allowing arbitraily large input datsets.
if false it will return an r2 value for each pair (this may be several billion r2 values)
'''
def batch_r2(gn,RamGB=15.0,fp16=True,ret_mean = False,r_squared = True):



    if fp16:
        cudatype = torch.cuda.HalfTensor
        cputype = torch.HalfTensor
        byte_per_snp = 38 #found from trial and error
    else:
        cudatype = torch.cuda.FloatTensor
        cputype = torch.FloatTensor
        byte_per_snp = 76  #found from trial and error


    #largest batch we can fit on GPU memory
    batchsize = int((RamGB * 1e9 / (gn.shape[1]*byte_per_snp)))

    starttime = time.time()
    y_shape = gn.shape[0]

    combinations_size = y_shape*(y_shape-1)//2
    print(combinations_size, "pairs")

    num_batches = np.ceil(combinations_size/batchsize).astype(int)

    if ret_mean:
        r2sum = 0
        r2size = 0
    else:
        #store returned r2 values in cpu RAM
        r2s = torch.zeros(0).type(cputype)


    for batch in tqdm(range(num_batches)):

        batchstart = batch*batchsize
        batchend = batchstart+batchsize

        #get pair combinations for current batch
        batchcomb = makecombs(y_shape,batchstart,batchend)

        xslice = gn[batchcomb[:,0]]
        yslice = gn[batchcomb[:,1]]

        #calcutlate r2 values
        if r_squared:
            r2 = (r_torch(xslice,yslice,cudatype)**2).cpu()
        else:
            r2 = r_torch(xslice,yslice,cudatype).cpu()



        if ret_mean:       
            #do running mean
            r2 = r2.float()
            nan_array = torch.isnan(r2) #remove Nan R Values
            r2 = r2[~ nan_array] #remove Nan R Values
            r2sum = r2sum + np.sum(r2.numpy())
            r2size = r2size + r2.shape[0]

        else:
            #add results to list
            r2s = torch.cat((r2s, r2), 0)

    print('Calculating R2 took ',time.time()-starttime,'Seconds')

    if ret_mean:
        return r2sum/r2size
    else:
        return r2s


In [0]:

import numpy as np
import glob
import pandas as pd


In [0]:
#LDNE: a program for estimating effective population size from data on linkage disequilibrium
#ROBIN  S. WAPLES and CHI DO 2008

def r2_to_Ne(r2,S):
    print("r2",r2)

    E = 1/S +3.19/(S**2)
    r2m = r2-E
    print("Num_Samples",S)

    # adding complex values as it avoids errors when tring to get square roots of negatives
    ne = (0.308+np.sqrt((0.308**2+0j)- 2.08*r2m)) / (2*r2m)
    print("ne_raw over 30:",ne)

    ne2 = ((1/3.)+np.sqrt((1/9.)- 2.76*r2m+0j)) / (2*r2m)
    print("ne_raw under 30:",ne2)

    nc =11
    pne_ovene = 0.098+0.219*np.log(nc)
    print("p over ne :",pne_ovene)

    print("Ne under 30:",ne2/pne_ovene)
    print("Ne over 30:",ne/pne_ovene)





In [0]:
#generate dummy data and run with mean
gn = torch.randint(-1, 3, (5000,100)).type(torch.cuda.CharTensor)
print(gn.shape)
r2 = batch_r2(gn,RamGB=15,fp16=False,ret_mean=True)


r2_to_Ne(r2,gn.shape[1])



In [0]:
#generate dummy data and run without mean and fp16
r2s = batch_r2(gn,RamGB=15,fp16=True,ret_mean=False)

nan_array = torch.isnan(r2s.float().cpu()) #remove Nan R Values
data = r2s[~ nan_array] #remove Nan R Values

print('std dev ',torch.std(data.float()).cpu().numpy())

r2m = np.mean(data.cpu().numpy())
r2_to_Ne(r2,gn.shape[1])

In [0]:
#import CSV

#currently set up for Samples on rows SNPs on columns

#Batch calc
datalist = glob.glob('*.csv') 
for dataset in datalist:
    
    df = pd.read_csv(dataset).fillna(-1)## fill na values with -1
    gn = torch.tensor(df[df.columns[1:]].values).T.type(torch.cuda.CharTensor) # discard non value entries and covert to cuda int8

    r2 = batch_r2(gn,RamGB=15,fp16=False,ret_mean=True)
    print ('-'*10,'Done with ',dataset,' ','-'*10)
    r2_to_Ne(r2,gn.shape[1])
    print ('-'*10,'-'*10)



In [0]:
#comparison test below
!pip install scikit-allel
import allel

In [0]:
#compare torch r to scikit allel

#generate dummy data
gn = torch.randint(-1, 3, (5000,100)).type(torch.cuda.CharTensor)


starttime = time.time()
rs = batch_r2(gn,RamGB=15,fp16=False,ret_mean=False,r_squared=False)
print('Torch r Took: ',time.time()-starttime,'Seconds')

starttime = time.time()
rorig =allel.rogers_huff_r(gn.cpu().numpy())
print('SciKit allele Took: ',time.time()-starttime,'Seconds')


rorigt = torch.tensor(rorig)

np.testing.assert_allclose(rorigt,rs,rtol=1e-5, atol=1e-5)
